# ROMS surface forcing netcdf check
**Author: Jun Sasaki, Coded on September 13, 2020, Updated on September 13, 2020**<br>
A sample code for manipulating `Projects/Sandy/romsforc_NARR.nc`

- See [ROMS Ocean Model](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html) 

In [ ]:
import numpy as np
from netCDF4 import num2date, date2num
import datetime
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import Stamen
from cartopy.io.img_tiles import OSM
from cartopy.mpl.ticker import LatitudeFormatter,LongitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import xarray as xr
import hvplot.xarray
import holoviews as hv
from holoviews import opts
import warnings
warnings.filterwarnings('ignore')
#GeoAxes._pcolormesh_patched = Axes.pcolormesh
%matplotlib inline

# Reading grid netcdf file
- See [Coordinate system of (xi, eta)](https://www.myroms.org/wiki/Fractional_Coordinate_System_(%CE%BE_-_%CE%B7_space))

In [ ]:
dir_path = "../../Projects/Sandy/"
grid_nc = dir_path + 'Sandy_roms_grid.nc'
#grid_nc = dir_path + 'training/Sandy_roms_grid.nc'
with xr.open_dataset(grid_nc) as grd:
    pass
grd

# Reading tidal forcing netcdf file

In [ ]:
surface_nc = dir_path + "romsforc_NARR_Sandy2012.nc"
#surface_nc = dir_path + "training/romsforc_NARR_Sandy2012.nc"
with xr.open_dataset(surface_nc) as ds:
    pass

In [ ]:
ds

In [ ]:
ds.coords['er']=ds['er']
ds['er'].attrs['long_name'] = 'e_rho'
ds.coords['xr']=ds['xr']
ds['xr'].attrs['long_name'] = 'x_rho'
ds

# Plot configuration

In [ ]:
z='Tair'  ### 'Uwind'/'Vwind'/'Pair'/'Tair'/'Qair'/'rain'/'swrad'/'lwrad'/'lwrad_down' 
dict_time={'Uwind':'wind_time', 'Vwind':'wind_time', 'Pair':'pair_time', \
           'Tair':'tair_time', 'Qair':'qair_time', 'rain':'rain_time', \
           'swrad':'srf_time', 'lwrad':'lrf_time', 'lwrad_down':'lrf_time'}
cmap='magma_r'
frame_height=200; frame_width=200
proj = ccrs.PlateCarree()

# Interactive plotting with [hvPlot](https://hvplot.holoviz.org/)
- To plot edge lines, `line_alpha` (0-1) in hvPlot should be a positive value.
- Set x-axis range: `xarray.Dataset.hvplot.quadmesh(xlim=(xmin, xmax))`
- Set num of color levels: `p.opts(opts.QuadMesh(color_levels=20))`
- Set value range: `xarray.Dataset.hvplot.quadmesh(clim=(cmin, cmax))`

In [ ]:
clim = (0, 28)  ### (cmin, cmax)/None
ds.hvplot.quadmesh(groupby=[dict_time[z]], x='xr', y='er',z=z , \
                   frame_height=frame_height, frame_width=frame_width, clim=clim, \
                   cmap=cmap, \
                   line_color='aqua', line_alpha=0.5, line_width=0.1).opts( \
                   opts.QuadMesh(color_levels=20))

## Plotting in (lon, lat)
- `rasterize=True` enhances a speed of plotting with time slider.
- `datashade=True` also enhances a speed of plotting with time slider; however `hover=True` does not work.

In [ ]:
clim = (0, 28)  ### (cmin, cmax) or None
ds.hvplot.quadmesh(groupby=[dict_time[z]], x='lon', y='lat',z=z , \
                   geo=True, coastline='10m', \
                   frame_height=frame_height, frame_width=frame_width, clim=clim, \
                   # line_color='aqua', line_alpha=0.5, line_width=0.1, \
                   # datashade=True, hover=True, \
                   cmap=cmap).opts( \
                   opts.QuadMesh(color_levels=20))

In [ ]:
clim = (0, 28)  ### (cmin, cmax) or None
ds.hvplot.quadmesh(groupby=[dict_time[z]], x='lon', y='lat',z=z , \
                   geo=True, coastline='10m', \
                   frame_height=frame_height, frame_width=frame_width, clim=clim, \
                   # line_color='aqua', line_alpha=0.5, line_width=0.1, \
                   # datashade=True, hover=True, \
                   rasterize=True, \
                   cmap=cmap).opts( \
                   opts.QuadMesh(color_levels=20))

### Gets available options for hvplot.quadmesh()
- Plot options are specified by `quadmesh().opts()`

In [ ]:
#hv.help(hv.QuadMesh)
#hvplot.help('quadmesh', generic=True, style=True)

# Plotting with matplotlib

In [ ]:
ds[z].isel(tair_time=0).plot(x='xr', y='er', extend='both', cmap='magma_r', \
                             vmin=0, vmax=28, \
                            )

## Specify time index without specifying time dimension name

In [ ]:
ds[z][0].plot(x='lon', y='lat', extend='both', cmap='magma_r', \
                             vmin=0, vmax=28, \
                            )

## Specify projection and manually plotting axes
- Axes ticks and labels do not appear unless manually specifying; thus this example becomes long.

In [ ]:
### Start settings by users
# Background map
#tiler = Stamen('terrain-background')
#tiler = OSM()
#zoom=2
extent = (-100, -70, 20, 50)  ### (lon_min, lon_max, lat_min, lat_max)
central_longitude = np.mean(extent[0:2])
# central_longitude =ds['lon'].mean().values
figsize = (6,4)
png = 'surface_forc.png'
## Default font size
plt.rcParams['font.size'] = 12
## Override axes label size
axes_label_size = 12
## Adjust surorunding margins
lon_min, lon_max = extent[0], extent[1]
lat_min, lat_max = extent[2], extent[3]
## Ticks intervals for lon and lat axes
dlon, dlat = (5, 5)
### End setting by users

xticks = np.arange(lon_min, lon_max, dlon)
yticks = np.arange(lat_min, lat_max, dlat)
proj = ccrs.PlateCarree(central_longitude=central_longitude)

fig = plt.figure(figsize=figsize)
ax = plt.axes(projection=proj)

ds[z][0].plot(ax=ax, x='lon', y='lat', extend='both', cmap='magma_r',\
              vmin=0, vmax=28, \
              transform=ccrs.PlateCarree(), \
              # alpha=1, edgecolor='aqua', linewidth=0.1\
             )
ax.set_extent([lon_min,lon_max,lat_min,lat_max], crs=ccrs.PlateCarree())
gl=ax.gridlines(draw_labels=True, xlocs=xticks, ylocs=yticks, linestyle=':', linewidth=1, color='k', alpha=0.8)
gl.right_labels=False
gl.top_labels=False
gl.xlabel_style={'size':axes_label_size}
gl.ylabel_style={'size':axes_label_size}
gl.xlocator = mticker.FixedLocator(xticks)
#ax.add_image(tiler=tiler, zoom=zoom)
ax.coastlines()
fig.savefig(png, dpi=300, bbox_inches='tight')

## Similar to the above but using xhycom.utils